In [31]:
import RNA  # ViennaRNA Python package
import pandas as pd

# Example DataFrame with sequences
data = {'siRNA': ['A1', 'A2', 'A3'], 
        'sequence': ['CUAAUAUGUUAAUUGAUUUAU', 'AAUAUGUUAAUUGAUUUAUAC', 'GAUUUAUACAAUUCCUUUCAA']}
df = pd.DataFrame(data)

def calculate_structure_and_energy(sequence):
    # Ensure the sequence is in uppercase
    sequence = sequence.upper()
    # Create a ViennaRNA fold compound
    fc = RNA.fold_compound(sequence)
    # Compute the MFE and structure
    structure, mfe = fc.mfe()
    return structure, mfe

# Apply the function to the DataFrame and store the results
df['structure'], df['mfe'] = zip(*df['sequence'].apply(calculate_structure_and_energy))

# Display the DataFrame
print(df)


  siRNA               sequence              structure  mfe
0    A1  CUAAUAUGUUAAUUGAUUUAU  .....................  0.0
1    A2  AAUAUGUUAAUUGAUUUAUAC  .....................  0.0
2    A3  GAUUUAUACAAUUCCUUUCAA  .....................  0.0


In [35]:
import pandas as pd

# Load efficacy data
efficacy_df = pd.read_csv("sirna_mrna_efficacy.csv")

# Load siRNA sequences from the FASTA file
from Bio import SeqIO

sirna_sequences = []
with open("sirna_1.fas", "r") as fasta_file:
    for record in SeqIO.parse(fasta_file, "fasta"):
        sirna_sequences.append({"siRNA": record.id, "sequence": str(record.seq).upper()})

sirna_df = pd.DataFrame(sirna_sequences)

# Merge siRNA sequences into the efficacy dataset
merged_df = pd.merge(sirna_df, efficacy_df, on="siRNA", how="inner")

# Reorder columns to place 'sequence' between 'siRNA' and 'mRNA'
columns = ["siRNA", "sequence", "mRNA", "efficacy"]
merged_df = merged_df[columns]

# Save or display the updated dataset
print(merged_df.head())


  siRNA               sequence      mRNA  efficacy
0    A1  CUAAUAUGUUAAUUGAUUUAU  BD135193     0.462
1    A2  AAUAUGUUAAUUGAUUUAUAC  BD135193     0.384
2    A3  GAUUUAUACAAUUCCUUUCAA  BD135193     0.514
3    A4  CAAUUCCUUUCAAUUUUAUCU  BD135193     0.364
4    A5  CAGACCAAAAUUAAAUAAGAA  BD135193     0.522


In [40]:
def calculate_gc_content(sequence):
    """Calculate GC content as a percentage."""
    gc_count = sequence.count("G") + sequence.count("C")
    return (gc_count / len(sequence)) * 100

def calculate_features(row):
    sequence = row["sequence"]
    antisense = sequence[::-1]  # Reverse sequence for antisense strand
    
    features = {
        "gc_content": calculate_gc_content(sequence),
        "rule_1_gc_range": 35 <= calculate_gc_content(sequence) <= 73,
        "rule_2_u_position_1": antisense[-1] == "U",
        "rule_3_a_positions_1_10": antisense[-1] == "A" and antisense[-10] == "A",
        "rule_4_no_g_14_18": antisense[-14] != "G" and antisense[-18] != "G",
        "rule_5_no_a_19": antisense[-19] != "A",
        "rule_6_motifs_present": any(motif in antisense for motif in ["UCU", "UCCG"]),
        "rule_7_motifs_absent": not any(motif in antisense for motif in ["ACGA", "GCC", "GUGG"]),
        # Placeholder for Gibbs energy changes and folding:
        "high_gibbs_1_4": True,  # Dummy value
        "low_gibbs_18_19": True,  # Dummy value
        "avoid_folding": True,  # Dummy value
    }
    return features

# Apply the feature engineering
features = merged_df.apply(calculate_features, axis=1)
features_df = pd.DataFrame(features.tolist())

# Combine features with the original dataset
siRNA_data_with_features = pd.concat([merged_df, features_df], axis=1)

print(siRNA_data_with_features.head())

  siRNA               sequence      mRNA  efficacy  gc_content  \
0    A1  CUAAUAUGUUAAUUGAUUUAU  BD135193     0.462   14.285714   
1    A2  AAUAUGUUAAUUGAUUUAUAC  BD135193     0.384   14.285714   
2    A3  GAUUUAUACAAUUCCUUUCAA  BD135193     0.514   23.809524   
3    A4  CAAUUCCUUUCAAUUUUAUCU  BD135193     0.364   23.809524   
4    A5  CAGACCAAAAUUAAAUAAGAA  BD135193     0.522   23.809524   

   rule_1_gc_range  rule_2_u_position_1  rule_3_a_positions_1_10  \
0            False                False                    False   
1            False                False                     True   
2            False                False                    False   
3            False                False                    False   
4            False                False                    False   

   rule_4_no_g_14_18  rule_5_no_a_19  rule_6_motifs_present  \
0               True            True                  False   
1               True            True                  False   
2    

In [56]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import pandas as pd
from Bio import SeqIO

# ================================
# Step 1: Feature Engineering
# ================================
def calculate_gc_content(sequence):
    """Calculate GC content as a percentage."""
    gc_count = sequence.count("G") + sequence.count("C")
    return (gc_count / len(sequence)) * 100 if len(sequence) > 0 else 0

def calculate_features(row):
    """Apply feature engineering rules to a sequence."""
    sequence = row["sequence"]
    antisense = sequence[::-1]  # Reverse sequence for antisense strand
    sequence_length = len(sequence)

    features = {
        "gc_content": calculate_gc_content(sequence),
        "rule_1_gc_range": 35 <= calculate_gc_content(sequence) <= 73,
        "rule_2_u_position_1": antisense[-1] == "U" if sequence_length >= 1 else False,
        "rule_3_a_positions_1_10": (antisense[-1] == "A" and antisense[-10] == "A") if sequence_length >= 10 else False,
        "rule_4_no_g_14_18": (antisense[-14] != "G" and antisense[-18] != "G") if sequence_length >= 18 else False,
        "rule_5_no_a_19": antisense[-19] != "A" if sequence_length >= 19 else False,
        "rule_6_motifs_present": any(motif in antisense for motif in ["UCU", "UCCG"]),
        "rule_7_motifs_absent": not any(motif in antisense for motif in ["ACGA", "GCC", "GUGG"]),
        "high_gibbs_1_4": True,  # Placeholder
        "low_gibbs_18_19": True,  # Placeholder
        "avoid_folding": True,  # Placeholder
    }
    return features

def prepare_feature_data(df):
    """Apply feature engineering to all rows in the dataset."""
    features = df.apply(calculate_features, axis=1)
    features_df = pd.DataFrame(features.tolist())
    
    # Ensure all features are numeric
    for col in features_df.columns:
        features_df[col] = pd.to_numeric(features_df[col], errors='coerce').fillna(0)
    
    return pd.concat([df, features_df], axis=1)

# ================================
# Step 2: Graph Data Preparation
# ================================
def create_graph(row, feature_columns):
    """Create a graph from a single row of the dataset."""
    sequence = row["sequence"]
    label = row["efficacy"]

    # Node features: One-hot encoding of nucleotides (A, U, C, G)
    one_hot = {"A": [1, 0, 0, 0], "U": [0, 1, 0, 0], "C": [0, 0, 1, 0], "G": [0, 0, 0, 1]}
    x = torch.tensor([one_hot[base] for base in sequence], dtype=torch.float)

    # Edges: Connect adjacent nucleotides (bidirectional)
    edge_index = torch.tensor(
        [[i, i + 1] for i in range(len(sequence) - 1)] +
        [[i + 1, i] for i in range(len(sequence) - 1)],
        dtype=torch.long
    ).t().contiguous()

    # Global features (add unsqueeze here)
    global_features = torch.tensor(
        [row[col] for col in feature_columns],
        dtype=torch.float
    ).unsqueeze(0)  # Add batch dimension

    return Data(
        x=x,
        edge_index=edge_index,
        y=torch.tensor([label], dtype=torch.float).view(-1, 1),
        global_features=global_features
    )


def prepare_graph_data(df, feature_columns):
    """Convert the dataset into graph objects."""
    return [create_graph(row, feature_columns) for _, row in df.iterrows()]

# ================================
# Step 3: Define GNN Model
# ================================
class GNN(torch.nn.Module):
    def __init__(self, num_node_features, num_global_features, hidden_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc_global = torch.nn.Linear(num_global_features, hidden_dim)
        self.fc1 = torch.nn.Linear(hidden_dim + hidden_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, data.batch)

        global_features = F.relu(self.fc_global(data.global_features))

        combined = torch.cat([x, global_features], dim=1)
        combined = F.relu(self.fc1(combined))
        return self.fc2(combined)

# ================================
# Step 4: Training and Evaluation
# ================================
def train_model(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Main Execution
# ================================
# Load efficacy data
efficacy_df = pd.read_csv("sirna_mrna_efficacy.csv")

# Load siRNA sequences from the FASTA file
sirna_sequences = []
with open("sirna_1.fas", "r") as fasta_file:
    for record in SeqIO.parse(fasta_file, "fasta"):
        sirna_sequences.append({"siRNA": record.id, "sequence": str(record.seq).upper()})

sirna_df = pd.DataFrame(sirna_sequences)

# Merge siRNA sequences into the efficacy dataset
merged_df = pd.merge(sirna_df, efficacy_df, on="siRNA", how="inner")

# Reorder columns to place 'sequence' between 'siRNA' and 'efficacy'
columns = ["siRNA", "sequence", "efficacy"]
merged_df = merged_df[columns]

# Feature engineering
data_with_features = prepare_feature_data(merged_df)

# Extract global feature columns, excluding 'mRNA'
feature_columns = list(data_with_features.columns.difference(["sequence", "efficacy", "siRNA"]))

# Prepare graph data
graph_data = prepare_graph_data(data_with_features, feature_columns)

# DataLoader
loader = DataLoader(graph_data, batch_size=2, shuffle=True)

# Model, optimizer, and loss function
num_global_features = len(feature_columns)
model = GNN(num_node_features=4, num_global_features=num_global_features, hidden_dim=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training loop
for epoch in range(50):
    train_loss = train_model(model, loader, optimizer, criterion)
    print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}")


/Users/harsha/anaconda3/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.0553
Epoch 2, Loss: 0.0410
Epoch 3, Loss: 0.0405
Epoch 4, Loss: 0.0404
Epoch 5, Loss: 0.0409
Epoch 6, Loss: 0.0404
Epoch 7, Loss: 0.0415
Epoch 8, Loss: 0.0400
Epoch 9, Loss: 0.0407
Epoch 10, Loss: 0.0412
Epoch 11, Loss: 0.0400
Epoch 12, Loss: 0.0405
Epoch 13, Loss: 0.0404
Epoch 14, Loss: 0.0395
Epoch 15, Loss: 0.0399
Epoch 16, Loss: 0.0389
Epoch 17, Loss: 0.0398
Epoch 18, Loss: 0.0395
Epoch 19, Loss: 0.0393
Epoch 20, Loss: 0.0394
Epoch 21, Loss: 0.0402


KeyboardInterrupt: 